In [20]:
import os
import warnings
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, \
                         SerperDevTool

warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())


True

In [21]:
openai_api_key = os.getenv("OPENAPI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

### Event Planner
Create a crew of agents that is able to create an event or a meetup for us

### Tools

In [22]:
scrape_tool = ScrapeWebsiteTool()
search_tool = SerperDevTool()

### Agents

In [23]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [24]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [25]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [33]:
event_manager_agent = Agent(
    role="Manager that decides if the event is a go or not",
    goal="Decide if the event is a go or not",
    verbose=True,
    backstory=(
        "You are the manager and the final decision maker"
    )
)

#### Pydantic object for Venue

Create this object to let agents create instances of this object and populate them as they work. Convert fuzzy outputs to a well-defined data structure and then JSON

In [26]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

### Tasks
The first task to execute is the venue task, once the venue is defined. The marketing and logistics tasks can run in parallel. However in latest versions of CrewAI, the crew can't finish with more that one async task

In [27]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [28]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [35]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [36]:
signoff_task = Task(
    description="After getting all the information for the {event_topic} "
                "evaluate if it meets the requirements and that the venue " 
                "is appropriate. Also check that the logistics and marketing"
                "strattegies arecorrect. Make sure that the event is not out of budget",
    expected_output="Final sign-off on the event plan.",
    agent=event_manager_agent
)

### Event Data Object

In [37]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

### Crew

In [38]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent,
            event_manager_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task,
           signoff_task],
    
    verbose=True,
    process=Process.sequential
)

2024-08-18 13:26:36,785 - 18636 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [39]:
result = event_management_crew.kickoff(inputs=event_details)

 [2024-08-18 13:26:39][DEBUG]: == Working Agent: Venue Coordinator
 [2024-08-18 13:26:39][INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to find a venue in San Francisco that can accommodate a Tech Innovation Conference. First, I should search the internet for venues in San Francisco that are suitable for tech events.

Action: Search the internet
Action Input: {"search_query": "Tech conference venues in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Top 12 tech conferences in San Francisco for 2024 · Developer Week · Women Impact Tech · SaaS Connect · Cybersecurity and Cloud Congress · Startup ...
---
Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/